In [ ]:
import tensorflow as tf
from keras.layers import Conv2D, LeakyReLU, Input, MaxPooling2D, Activation, Concatenate, Dropout, Lambda, Add, UpSampling2D
from keras.models import Model, Sequential
import keras
import matplotlib.pyplot as plt
from tensorflow.data import AUTOTUNE
import uuid

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Get data set objects together

In [ ]:
#Environmnet stuff
monet_path = "/content/drive/MyDrive/Colab Projects/CycleGan/Raw/Monet"
image_path = "/content/drive/MyDrive/Colab Projects/CycleGan/Raw/Images"

ROOT_PATH = "/content/drive/MyDrive/Colab Projects/CycleGan"
MODEL_NAME = "LowFidelityGAN_V1_Decay"


In [ ]:
#Make the datasets
my_monet_ds_train, my_monet_ds_val = tf.keras.utils.image_dataset_from_directory(
    monet_path,
    validation_split=0.1,
    subset="both",
    seed=42,
    image_size=(128, 128),
    batch_size = 1,
    labels = None,
)

my_image_ds_train, my_image_ds_val = tf.keras.utils.image_dataset_from_directory(
    image_path,
    validation_split=0.1,
    subset="both",
    seed=42,
    image_size=(128, 128),
    batch_size = 1,
    labels = None,
)

DATASET_HEIGHT = max(len(my_monet_ds_train), len(my_image_ds_train))

Found 1193 files.
Using 1074 files for training.
Using 119 files for validation.
Found 7037 files.
Using 6334 files for training.
Using 703 files for validation.


In [ ]:
#Pre processing
def augment_and_normalize(image):
    image = tf.cast(image, tf.float32)  # convert from uint8
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=20)  # for 0–255 range
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    image = tf.clip_by_value(image, 0.0, 255.0)
    image = image / 127.5 - 1.0  # scale to [-1, 1]
    return image


In [ ]:

my_monet_ds_train = (
  my_monet_ds_train
  .map(augment_and_normalize, num_parallel_calls=AUTOTUNE)
  .cache()
  .shuffle(1000)
  .repeat()
  .prefetch(AUTOTUNE)
)

my_image_ds_train = (
  my_image_ds_train
  .map(augment_and_normalize, num_parallel_calls=AUTOTUNE)
  .cache()
  .shuffle(1000)
  .repeat()
  .prefetch(AUTOTUNE)
)

my_monet_ds_train = iter(my_monet_ds_train)
my_image_ds_train = iter(my_image_ds_train)

## Define an Instance norm Layer

In [ ]:
class InstanceNormalization(keras.layers.Layer):
    def __init__(self, epsilon=1e-5, **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        # One scale and bias per channel
        self.gamma = self.add_weight(
            shape=(input_shape[-1],),
            initializer='ones',
            trainable=True,
            name='gamma'
        )
        self.beta = self.add_weight(
            shape=(input_shape[-1],),
            initializer='zeros',
            trainable=True,
            name='beta'
        )
        super(InstanceNormalization, self).build(input_shape)

    def call(self, inputs):
        # Compute mean and variance per instance, per channel
        mean, variance = tf.nn.moments(inputs, axes=[1, 2], keepdims=True)
        normalized = (inputs - mean) / tf.sqrt(variance + self.epsilon)
        return self.gamma * normalized + self.beta

    def get_config(self):
        config = super().get_config().copy()
        config.update({"epsilon": self.epsilon})
        return config


## Put the Discrim model together

In [ ]:
def build_discriminator(input_shape=(128, 128, 3)):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(512, kernel_size=4, strides=1, padding='same')(x)
    x = InstanceNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(1, kernel_size=4, strides=1, padding='same')(x)

    return Model(inputs, x, name='PatchGAN_70x70')

my_patchgan_discriminator = build_discriminator()
my_patchgan_discriminator.summary()


Model: "PatchGAN_70x70"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 64, 64, 64)     │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 128)    │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ instance_normalization          │ (None, 32, 32, 128)    │           256 │
│ (InstanceNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 256)    │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ instance_normalization_1        │ (None, 16, 16, 256)    │           512 │
│ (InstanceNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 512)    │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ instance_normalization_2        │ (None, 16, 16, 512)    │         1,024 │
│ (InstanceNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 1)      │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,766,529 (10.55 MB)

 Trainable params: 2,766,529 (10.55 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def build_weak_discriminator(input_shape=(128, 128, 3)):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(1, kernel_size=4, strides=1, padding='same')(x)

    return Model(inputs, x, name='PatchGAN_70x70')

my_patchgan_discriminator = build_discriminator()
my_patchgan_discriminator.summary()


Model: "PatchGAN_70x70"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 64)     │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 128)    │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ instance_normalization_3        │ (None, 32, 32, 128)    │           256 │
│ (InstanceNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 256)    │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ instance_normalization_4        │ (None, 16, 16, 256)    │           512 │
│ (InstanceNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 512)    │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ instance_normalization_5        │ (None, 16, 16, 512)    │         1,024 │
│ (InstanceNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 16, 1)      │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,766,529 (10.55 MB)

 Trainable params: 2,766,529 (10.55 MB)

 Non-trainable params: 0 (0.00 B)

## Generator model

In [ ]:
# === Reflection Padding ===
def ReflectionPad2D(pad):
    return Lambda(lambda x: tf.pad(x, [[0, 0], [pad, pad], [pad, pad], [0, 0]], mode='REFLECT'))

In [ ]:
# === Residual Block ===
def ResidualBlock(filters, kernel_size=3):
    pad = kernel_size // 2
    def block(x):
        y = ReflectionPad2D(pad)(x)
        y = Conv2D(filters, kernel_size, padding='valid')(y)
        y = InstanceNormalization()(y)
        y = Activation('relu')(y)

        y = ReflectionPad2D(pad)(y)
        y = Conv2D(filters, kernel_size, padding='valid')(y)
        y = InstanceNormalization()(y)

        return Add()([x, y])
    return block
##No activation at the end ?!

In [ ]:
def Generator(input_shape=(128, 128, 3), n_res_blocks=6):
    inputs = Input(shape=input_shape)

    # Encoder
    x = ReflectionPad2D(3)(inputs)
    x = Conv2D(64, kernel_size=7, padding='valid')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)

    # Residual Blocks
    for _ in range(n_res_blocks):
        x = ResidualBlock(256)(x)

    # Decoder
    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    x = Conv2D(128, kernel_size=3, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    x = Conv2D(64, kernel_size=3, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)

    x = ReflectionPad2D(3)(x)
    x = Conv2D(3, kernel_size=7, padding='valid', activation='tanh')(x)

    return Model(inputs, x, name='JohnsonGenerator')

# Example:
gen = Generator(input_shape=(128,128,3))
gen.summary()

Model: "JohnsonGenerator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 134, 134,  │          0 │ input_layer_2[0]… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 128, 128,  │      9,472 │ lambda[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ instance_normaliza… │ (None, 128, 128,  │        128 │ conv2d_10[0][0]   │
│ (InstanceNormaliza… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128, 128,  │          0 │ instance_normali… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 64, 64,    │     73,856 │ activation[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ instance_normaliza… │ (None, 64, 64,    │        256 │ conv2d_11[0][0]   │
│ (InstanceNormaliza… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 64, 64,    │          0 │ instance_normali… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 32, 32,    │    295,168 │ activation_1[0][… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ instance_normaliza… │ (None, 32, 32,    │        512 │ conv2d_12[0][0]   │
│ (InstanceNormaliza… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 32, 32,    │          0 │ instance_normali… │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 34, 34,    │          0 │ activation_2[0][… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 32, 32,    │    590,080 │ lambda_1[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ instance_normaliza… │ (None, 32, 32,    │        512 │ conv2d_13[0][0]   │
│ (InstanceNormaliza… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 32, 32,    │          0 │ instance_normali… │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 34, 34,    │          0 │ activation_3[0][… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 32, 32,    │    590,080 │ lambda_2[0][0]  

 Total params: 7,845,123 (29.93 MB)

 Trainable params: 7,845,123 (29.93 MB)

 Non-trainable params: 0 (0.00 B)

## Loss functions

In [ ]:

#L2 Loss function
def my_square_loss(y_true, y_pred):
    y_true = tf.cast(y_true, y_pred.dtype)
    return tf.reduce_mean(tf.square(y_true - y_pred))

#Generator loss function
def generator_loss(descrim_output):
        return tf.reduce_mean(tf.math.squared_difference(tf.ones_like(descrim_output), descrim_output))


## Image buffer to stabilise training

In [ ]:
import numpy as np

class ImageBuffer:
    """
    Implements a buffer that stores previously generated images.
    When queried, it returns a mix of old and new images.
    """
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.data = []

    def query(self, image):
        if len(self.data) < self.max_size:
            # Buffer not full: store and return current fake
            self.data.append(image)
            return image
        else:
            if np.random.rand() > 0.5:
                # Use buffer: pick old, swap with current
                idx = np.random.randint(0, self.max_size)
                old_image = self.data[idx]
                self.data[idx] = image
                return old_image
            else:
                # Use current fake
                return image


## Add in decay schedule object

In [ ]:
class LinearDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, decay_start_epoch, total_epochs, steps_per_epoch):
        super().__init__()
        self.initial_lr = initial_lr
        self.decay_start_epoch = tf.cast(decay_start_epoch, tf.float32)
        self.total_epochs = tf.cast(total_epochs, tf.float32)
        self.steps_per_epoch = tf.cast(steps_per_epoch, tf.float32)

    def __call__(self, step):
        epoch = tf.cast(step, tf.float32) / self.steps_per_epoch
        lr = tf.cond(
            epoch < self.decay_start_epoch,
            lambda: self.initial_lr,
            lambda: self.initial_lr * (1.0 - (epoch - self.decay_start_epoch) / (self.total_epochs - self.decay_start_epoch))
        )
        return tf.maximum(lr, 0.0)


## Adding Alternating trainin

In [ ]:
def to_display(img):
    return (img[0] + 1) / 2 #* 0.5 + 0.5#tf.clip_by_value(img * 0.5 + 0.5, 0.0, 1.0)

In [ ]:
from keras.optimizers import Adam
import numpy as np
import time

# === CONFIG ===
NUM_EPOCHS = 100
EPOCH_LENGTH = DATASET_HEIGHT
INITIAL_LR = 2e-4
DECAY_START = 1

# === TRACKERS ===
current_epoch = tf.Variable(0, dtype=tf.int64)
global_step = tf.Variable(0, dtype=tf.int64)

DISPLAY_INTERVAL = 500  # record loss every 5 steps

# Optimizers for all models
monet_gen_opt = Adam(0.0, beta_1=0.5)
photo_gen_opt = Adam(0.0, beta_1=0.5)
monet_disc_opt = Adam(0.0, beta_1=0.5)
photo_disc_opt = Adam(0.0, beta_1=0.5)

# Initialize models
monet_generator = Generator((128, 128, 3))
photo_generator = Generator((128, 128, 3))
monet_discriminator = build_weak_discriminator()
photo_discriminator = build_discriminator()

#eg_image = next(my_image_ds_train)
#eg_output = monet_generator(eg_image)

# Create buffers for each fake domain
monet_fake_buffer = ImageBuffer(max_size=50)
photo_fake_buffer = ImageBuffer(max_size=50)


In [ ]:
monet_generator.load_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_monet_generator.weights.h5")
photo_generator.load_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_photo_generator.weights.h5")
monet_discriminator.load_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_monet_discriminator.weights.h5")
photo_discriminator.load_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_photo_discriminator.weights.h5")

In [ ]:

@tf.function
def train_step(monet_image, photo_image, bool_val):
    with tf.GradientTape(persistent=True) as tape:
        # === Generate fakes ===
        fake_monet = monet_generator(photo_image, training=True)
        fake_photo = photo_generator(monet_image, training=True)

        # === Cycle back ===
        cycled_photo = photo_generator(fake_monet, training=True)
        cycled_monet = monet_generator(fake_photo, training=True)

        # === Identity mapping ===
        same_monet = monet_generator(monet_image, training=True)
        same_photo = photo_generator(photo_image, training=True)

        # === Discriminator real ===
        disc_real_monet = monet_discriminator(monet_image, training=True)
        disc_real_photo = photo_discriminator(photo_image, training=True)

        # === Use buffer for discriminator fakes ===
        fake_monet_for_disc = monet_fake_buffer.query(fake_monet)
        fake_photo_for_disc = photo_fake_buffer.query(fake_photo)

        disc_fake_monet = monet_discriminator(fake_monet_for_disc, training=True)
        disc_fake_photo = photo_discriminator(fake_photo_for_disc, training=True)

        # === Losses ===
        monet_disc_loss = 0.5 * (
            my_square_loss(tf.ones_like(disc_real_monet), disc_real_monet) +
            my_square_loss(tf.zeros_like(disc_fake_monet), disc_fake_monet)
        )
        photo_disc_loss = 0.5 * (
            my_square_loss(tf.ones_like(disc_real_photo), disc_real_photo) +
            my_square_loss(tf.zeros_like(disc_fake_photo), disc_fake_photo)
        )

        monet_gen_loss = my_square_loss(tf.ones_like(
            monet_discriminator(fake_monet, training=True)
        ), monet_discriminator(fake_monet, training=True))
        photo_gen_loss = my_square_loss(tf.ones_like(
            photo_discriminator(fake_photo, training=True)
        ), photo_discriminator(fake_photo, training=True))

        cycle_loss = tf.reduce_mean(tf.abs(photo_image - cycled_photo)) + \
                     tf.reduce_mean(tf.abs(monet_image - cycled_monet))

        identity_loss = tf.reduce_mean(tf.abs(monet_image - same_monet)) + \
                        tf.reduce_mean(tf.abs(photo_image - same_photo))

        total_monet_gen_loss = monet_gen_loss + 10 * cycle_loss + 5 * identity_loss
        total_photo_gen_loss = photo_gen_loss + 10 * cycle_loss + 5 * identity_loss

    # === Apply gradients ===
    monet_disc_grads = tape.gradient(monet_disc_loss, monet_discriminator.trainable_variables)
    photo_disc_grads = tape.gradient(photo_disc_loss, photo_discriminator.trainable_variables)
    monet_gen_grads = tape.gradient(total_monet_gen_loss, monet_generator.trainable_variables)
    photo_gen_grads = tape.gradient(total_photo_gen_loss, photo_generator.trainable_variables)

    if bool_val:
      monet_disc_opt.apply_gradients(zip(monet_disc_grads, monet_discriminator.trainable_variables))
      photo_disc_opt.apply_gradients(zip(photo_disc_grads, photo_discriminator.trainable_variables))

    monet_gen_opt.apply_gradients(zip(monet_gen_grads, monet_generator.trainable_variables))
    photo_gen_opt.apply_gradients(zip(photo_gen_grads, photo_generator.trainable_variables))

    return (
        monet_disc_loss, photo_disc_loss,
        monet_gen_loss, photo_gen_loss,
        cycle_loss, identity_loss
    )

In [ ]:
def setup_cyclegan_checkpoint():
    """
    Sets up CycleGAN checkpoint manager using global models & ALL FOUR optimizers.
    Uses these global variables:
      monet_generator, photo_generator,
      monet_discriminator, photo_discriminator,
      monet_gen_opt, photo_gen_opt, monet_disc_opt, photo_disc_opt
    """
    global ckpt, ckpt_manager, current_epoch  # make them global

    # Ensure epoch is a tf.Variable if not already set
    if 'current_epoch' not in globals():
        current_epoch = tf.Variable(1, name="epoch", trainable=False)

    ckpt = tf.train.Checkpoint(
        monet_generator=monet_generator,
        photo_generator=photo_generator,
        monet_discriminator=monet_discriminator,
        photo_discriminator=photo_discriminator,
        monet_gen_opt=monet_gen_opt,
        photo_gen_opt=photo_gen_opt,
        monet_disc_opt=monet_disc_opt,
        photo_disc_opt=photo_disc_opt,
        epoch=current_epoch
    )

    ckpt_manager = tf.train.CheckpointManager(
        ckpt,
        f"{ROOT_PATH}/checkpoints/{MODEL_NAME}",
        max_to_keep=10
    )

    if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
        print(f"✅ Restored from {ckpt_manager.latest_checkpoint} (epoch {int(current_epoch.numpy())})")
    else:
        print("🚀 Training from scratch.")


In [ ]:
print(ROOT_PATH)
print(MODEL_NAME)

%load_ext tensorboard
logdir = f"{ROOT_PATH}/logs/{MODEL_NAME}"
%tensorboard --logdir "$logdir" --port 0  # let Colab pick a port

In [ ]:
running_loss = []

from datetime import datetime
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = f"{ROOT_PATH}/logs/{MODEL_NAME}/{current_time}DescrimOn3"
summary_writer = tf.summary.create_file_writer(log_dir)


dummy = tf.zeros([1, 128, 128, 3])
_ = monet_generator(dummy)
_ = photo_generator(dummy)
_ = monet_discriminator(dummy)
_ = photo_discriminator(dummy)


lr_schedule = LinearDecay(INITIAL_LR, DECAY_START, NUM_EPOCHS, EPOCH_LENGTH)

monet_gen_opt = Adam(learning_rate=lr_schedule, beta_1=0.5)
photo_gen_opt = Adam(learning_rate=lr_schedule, beta_1=0.5)
monet_disc_opt = Adam(learning_rate=lr_schedule, beta_1=0.5)
photo_disc_opt = Adam(learning_rate=lr_schedule, beta_1=0.5)

setup_cyclegan_checkpoint()

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    epoch_losses = []
    start_time = time.time()

    if (epoch + 1) % 5 == 0:
        save_path = ckpt_manager.save()
        print(f"✅ Saved checkpoint for epoch {epoch + 1}: {save_path}")

        monet_generator.save_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_monet_generator.weights.h5")
        photo_generator.save_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_photo_generator.weights.h5")
        monet_discriminator.save_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_monet_discriminator.weights.h5")
        photo_discriminator.save_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_photo_discriminator.weights.h5")


    for step in range(EPOCH_LENGTH):
        monet_image = next(my_monet_ds_train)
        photo_image = next(my_image_ds_train)

        losses = train_step(monet_image, photo_image, ((step % 3) == 0))

        global_step.assign_add(1)

        loss_vals = [tf.squeeze(l).numpy() for l in losses]

        #global_step = epoch * EPOCH_LENGTH + step

        # Add to running buffer
        running_loss.append(loss_vals)

        if step % DISPLAY_INTERVAL == 0:
            avg_running_loss = np.mean(running_loss, axis=0)

            with summary_writer.as_default():
                tf.summary.scalar("Avg100/M_disc", avg_running_loss[0], step=global_step)
                tf.summary.scalar("Avg100/P_disc", avg_running_loss[1], step=global_step)
                tf.summary.scalar("Avg100/M_gen", avg_running_loss[2], step=global_step)
                tf.summary.scalar("Avg100/P_gen", avg_running_loss[3], step=global_step)
                tf.summary.scalar("Avg100/Cycle", avg_running_loss[4], step=global_step)
                tf.summary.scalar("Avg100/Identity", avg_running_loss[5], step=global_step)
                tf.summary.scalar("LearningRate", lr_schedule(global_step), step=global_step)

                # 1️⃣  Get real samples
                real_photo = photo_image
                real_monet = monet_image

                # 2️⃣  Generate mappings
                generated_monet = monet_generator(real_photo, training=False)
                generated_photo = photo_generator(real_monet, training=False)

                # 3️⃣  Convert to displayable (your existing function)
                real_photo_disp = to_display(real_photo)
                generated_monet_disp = to_display(generated_monet)
                real_monet_disp = to_display(real_monet)
                generated_photo_disp = to_display(generated_photo)

                # 4️⃣  Stack into a 2x2 grid (rows)
                row1 = tf.concat([real_photo_disp, generated_monet_disp], axis=1)
                row2 = tf.concat([real_monet_disp, generated_photo_disp], axis=1)
                grid = tf.concat([row1, row2], axis=0)

                # 5️⃣  Make sure it's float32 and has batch dimension
                grid = tf.convert_to_tensor(grid, dtype=tf.float32)
                grid = tf.expand_dims(grid, 0)  # add batch dimension

                # 6️⃣  Log it as a single image
                tf.summary.image("Grid/2x2_Examples", grid, step=global_step)


            running_loss = []  # reset for next 100 steps
        print(f"\rStep {step+1}/{EPOCH_LENGTH}", end='')
    current_epoch.assign_add(1)

    #avg_losses = np.mean(epoch_losses, axis=0)
    print(f"\nEpoch {epoch + 1} done in {int(time.time()-start_time)}s.")
    #all_losses.extend(epoch_losses)


✅ Restored from /content/drive/MyDrive/Colab Projects/CycleGan/checkpoints/LowFidelityGAN_V1_Decay/ckpt-9 (epoch 43)

Epoch 1/100
Step 6334/6334
Epoch 1 done in 700s.

Epoch 2/100
Step 6334/6334
Epoch 2 done in 527s.

Epoch 3/100
Step 6334/6334
Epoch 3 done in 527s.

Epoch 4/100
Step 6334/6334
Epoch 4 done in 527s.

Epoch 5/100
✅ Saved checkpoint for epoch 5: /content/drive/MyDrive/Colab Projects/CycleGan/checkpoints/LowFidelityGAN_V1_Decay/ckpt-10
Step 6334/6334
Epoch 5 done in 537s.

Epoch 6/100
Step 6334/6334
Epoch 6 done in 527s.

Epoch 7/100
Step 6334/6334
Epoch 7 done in 526s.

Epoch 8/100
Step 6334/6334
Epoch 8 done in 527s.

Epoch 9/100
Step 6334/6334
Epoch 9 done in 526s.

Epoch 10/100
✅ Saved checkpoint for epoch 10: /content/drive/MyDrive/Colab Projects/CycleGan/checkpoints/LowFidelityGAN_V1_Decay/ckpt-11
Step 6334/6334
Epoch 10 done in 531s.

Epoch 11/100
Step 6334/6334
Epoch 11 done in 526s.

Epoch 12/100
Step 6334/6334
Epoch 12 done in 526s.

Epoch 13/100
Step 6334/6334
E

In [ ]:
save_path = ckpt_manager.save()
#print(f"✅ Saved checkpoint for epoch {epoch + 1}: {save_path}")

monet_generator.save_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_monet_generator.weights.h5")
photo_generator.save_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_photo_generator.weights.h5")
monet_discriminator.save_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_monet_discriminator.weights.h5")
photo_discriminator.save_weights(f"{ROOT_PATH}/models/{MODEL_NAME}_photo_discriminator.weights.h5")

In [ ]:
for i in range(10):
  photo_image = next(my_image_ds_train)
  test_photo_to_monet = monet_generator(photo_image, training=False)
  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.imshow(to_display(photo_image))
  plt.title('Input Photo')
  plt.subplot(1, 2, 2)
  plt.imshow(to_display(test_photo_to_monet))
  plt.title('Generated Monet')
  plt.show()
  plt.close()